In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from libs import schema_handler as sh
from libs import datafunctions as spdf
from libs import utils as ut
from libs import azureauth as az

In [0]:
silver_table = dbutils.widgets.get('silver_table')
gold_table = dbutils.widgets.get('gold_table')
source = dbutils.widgets.get('source')
watermark = dbutils.widgets.get('watermark')

In [0]:
env = az.get_env(spark)
watermark_timestamp = "DH_"+silver_table.split('_')[1].upper()+"_"+ut.format_timestamp(watermark)
field_schema, primary_keys = sh.read_schema(env = env, layer= "gold", source= source, entity=gold_table)
surrogate_keys = [field for field in sh.extract_attributes(field_schema) if "_sk" in field.lower()]
#surrogate_keys = sh.extract_surrogate_keys(field_schema)
print(surrogate_keys)

In [0]:
dfSilver = spark.read.table(f"datahub_{env}_silver.{source}.{silver_table}").filter(F.col("extract_timestamp") > watermark_timestamp)
display(dfSilver)